<a href="https://colab.research.google.com/github/tejaswishetty17/GenAI_Work/blob/main/Conversational_RAG_Solutionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.2 MB/s eta 0:00:

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

In [4]:
import os
os.environ['OPENAI_API_KEY'] = "<YOUR_API_KEY>"


In [11]:
#Step 1: Load & Split Documents
loader = TextLoader("/content/sample_data/langchain_doc.txt")
raw_docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=100)
docs = splitter.split_documents(raw_docs)
len(docs)

20

In [6]:
#Step 2: Embed & Store in Chroma DB
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")
persist_dir = ".\chroma_store_chat2"
vectorestore = Chroma.from_documents(docs, embedding = embedding_model, persist_directory=persist_dir)
retriever = vectorestore.as_retriever(search_kwargs = {"k":3})

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert on Langchain. Use the provided context to answer user questions accurately."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "here is some context to help you:\n\n{context}\n\n Now answer this question:\n{question}")
])


In [10]:
#Step 3 : Define the RAG chain
def join_chunks(chunks):
  return "\n\n".join(doc.page_content for doc in chunks)

retrieval_chain = itemgetter("question") | retriever | join_chunks

full_chain = RunnableParallel(
    question = itemgetter("question"),
    context = retrieval_chain,
    chat_history = itemgetter("chat_history")
) | prompt | ChatOpenAI(model = "gpt-4o-mini") | StrOutputParser()

In [15]:
#Step 4: Run Chat Loop
chat_history = []
print("Chatbot: Hello! Ask me anything related to Langchain. Type 'exit' to quit. \n")

while True:
  user_input = input("You: ")
  if user_input.lower() == "exit":
    print("Chatbot: Goodbye! Have a great day!")
    break

    #Prepare input with chat history and question
  inputs = {
      "question":user_input,
      "chat_history": chat_history
      }

  #Get RAG-based response
  response = full_chain.invoke(inputs)
  print(f"Chatbot: {response}\n")

  #Maintain structured message history
  chat_history.append(HumanMessage(content=user_input))
  chat_history.append(AIMessage(content=response))


Chatbot: Hello! Ask me anything related to Langchain. Type 'exit' to quit. 

You: what is langchain
Chatbot: LangChain is a framework designed to facilitate the development of applications that utilize language models. Its core philosophy revolves around providing modular abstractions, prebuilt utilities, and community-backed best practices to streamline the integration of third-party APIs, document handling, and agent-based solutions. LangChain enables developers to efficiently implement solutions for tasks such as data extraction and transformation, as well as working with various document formats like PDFs and Markdown. Overall, LangChain offers comprehensive support for building and deploying language model applications.

You: Can you tell a bit more on above
Chatbot: Certainly! LangChain provides a comprehensive framework that helps developers efficiently build applications leveraging language models. Here’s a deeper look into the specific concepts mentioned:

1. **Handling Long D